In [1]:
from Bio import SeqIO
import pandas as pd
import re
import os

In [2]:
def aa_3di_convert(aafile, al3di, alout):
    
    unalaa_dic = SeqIO.to_dict(SeqIO.parse(aafile, 'fasta'))

    aaalseq = ''

    for rec in SeqIO.parse(al3di, 'fasta'):
        c=0
        diseq = str(rec.seq)
        alaa = ''
        for char in diseq:
            if char == '-':
                alaa = alaa + '-'
            else:
                alaa = alaa + str(unalaa_dic[rec.id].seq)[c]
                c = c+1

        aaalseq = aaalseq + '>%s\n%s\n'%(rec.id, alaa)

    with open(alout, 'w') as out:
        out.write(aaalseq[:-1])

In [6]:
def aa_3di_comb(al3di, alaa, alout):

    al3di_dic = SeqIO.to_dict(SeqIO.parse(al3di, 'fasta'))
    alaa_dic = SeqIO.to_dict(SeqIO.parse(alaa, 'fasta'))
    
    #combine the alignments
    combfas =  '\n'.join([ '>%s\n%s%s'%(x, str(al3di_dic[x].seq), str(alaa_dic[x].seq)) for x in list(alaa_dic)])
    
    with open(alout, 'w') as out:
         out.write(combfas)
            
    
    #make iqtree partition nexus file
    len3di = len(al3di_dic[list(al3di_dic)[0]].seq)
    lenaa = len(alaa_dic[list(alaa_dic)[0]].seq)
    nxsstr = "#nexus\nbegin sets;\n\tcharset part1 = 1-%i;\n\tcharset part2 = %i-%i;\n\tcharpartition mine = 3DI+F+R6:part1, LG+F+R6:part2;\nend;"%(len3di, len3di+1, len3di+lenaa)
    with open(alout.replace('.fas', '_parts.nex'), 'w') as out:
        out.write(nxsstr)
        

In [9]:
def pass_trim(totrimal, cols, outf):
        
    newtrim = ''
    
    for rec in SeqIO.parse(totrimal, 'fasta'):
        
        trimseq = ''.join([str(rec.seq)[i] for i in range(len(rec.seq)) if i in cols])
        newtrim = newtrim + '>%s\n%s\n'%(rec.id, trimseq)
        
    with open(outf, 'w') as out:
        out.write(newtrim[:-1])

<br>
<br>

Infer 3di alignment with `famsa3di` and `foldmason`

Create 3di-based aa alignment for `famsa3di` (foldmason makes this automatically):

In [3]:
aa_3di_convert('RSV_F_6APB_A_MODEL_1_aa.fa',
               'RSV_F_6APB_A_MODEL_1_famsa3di_3di.fa',
               'RSV_F_6APB_A_MODEL_1_famsa3di_aa.fa' )

<br>

Trim with `trimal` `-gt 0.35`

Pass trim to aa alignment:

In [10]:
famsatrim35 = [1413, 1414, 1415, 1416, 1417, 1418, 1438, 1441, 1442, 1443, 1451, 1452, 1453, 1459, 1470, 1471, 1472, 1473, 1481, 1482, 1483, 1489, 1490, 1491, 1512, 1513, 1514, 1529, 1530, 1549, 1550, 1551, 1552, 1553, 1554, 1563, 1564, 1565, 1570, 1579, 1580, 1581, 1582, 1583, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1608, 1609, 1610, 1633, 1634, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663, 1664, 1685, 1692, 1693, 1694, 1695, 1696, 1713, 1714, 1715, 1728, 1729, 1730, 1731, 1732, 1733, 1734, 1741, 1742, 1745, 1746, 1747, 1772, 1773, 1774, 1775, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1804, 1805, 1806, 1807, 1808, 1809, 1817, 1818, 1819, 1820, 1821, 1825, 1826, 1827, 1828, 1829, 1841, 1842, 1843, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1884, 1897, 1898, 1899, 1912, 1913, 1925, 1926, 1936, 1953, 1954, 1960, 1961, 1962, 1963, 1964, 1981, 1982, 1983, 1984, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2032, 2033, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2149, 2150, 2151, 2152, 2153, 2162, 2163, 2174, 2175, 2176, 2177, 2178, 2197, 2198, 2199, 2200, 2201, 2202, 2225, 2226, 2233, 2234, 2235, 2236, 2237, 2260, 2261, 2281, 2282, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 2317, 2337, 2338, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2375, 2376, 2377, 2378, 2379, 2380, 2381, 2382, 2396, 2397, 2398, 2399, 2400, 2401, 2402, 2403, 2404, 2405, 2409, 2416, 2431, 2432, 2433, 2434, 2435, 2436, 2458, 2459, 2460, 2467, 2468, 2469, 2470, 2471, 2483, 2484, 2485, 2486, 2487, 2495, 2503, 2504, 2505, 2509, 2510, 2511, 2512, 2513, 2514, 2516, 2517, 2525, 2526, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2545, 2546, 2551, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565, 2566, 2578, 2579, 2580, 2581, 2599, 2614, 2615, 2616, 2617, 2618, 2630, 2631, 2632, 2633, 2639, 2640, 2641, 2642, 2643, 2644, 2655, 2656, 2657, 2665, 2672, 2673, 2674, 2675, 2690, 2691, 2692, 2712, 2713, 2714, 2715, 2716, 2717, 2731, 2732, 2736, 2737, 2738, 2739, 2740, 2744, 2745, 2765, 2766, 2767, 2768, 2769, 2770, 2771, 2772, 2780, 2781, 2782, 2787, 2788, 2789, 2790, 2791, 2793, 2794, 2795, 2798, 2799, 2802, 2803, 2811, 2815, 2816, 2823, 2824, 2830, 2831, 2832, 2833, 2834, 2835, 2836, 2840, 2844, 2845, 2849, 2853, 2854, 2855, 2856, 2857, 2858, 2863, 2864, 2865, 2866, 2869, 2870, 2871, 2872, 2873, 2874, 2875, 2879, 2886, 2891, 2894, 2895, 2896, 2897, 2898, 2899, 2902, 2911, 2915, 2916, 2925, 2931, 2932, 2933, 2934, 2936, 2937, 2938, 2939, 2940, 2950, 2951, 2952, 2953, 2954, 2955, 2956, 2957, 2958, 2961, 2970, 2971, 2972, 2975, 2980, 2981, 2982, 2983, 2984, 2985, 2995, 2996, 2997, 2998, 2999, 3000, 3001, 3002, 3009, 3020, 3033, 3037, 3043, 3044, 3045, 3046, 3047, 3048, 3059, 3066, 3067, 3068, 3082, 3083, 3084, 3085, 3086, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3151, 3152, 3165, 3166, 3167, 3168, 3169, 3170, 3171, 3172, 3180, 3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190, 3191, 3192, 3207, 3208, 3209, 3210, 3211, 3237, 3238, 3239, 3240, 3251, 3252, 3253, 3254, 3268, 3269, 3270, 3271, 3272, 3273, 3274, 3276, 3277, 3278, 3279, 3280, 3281, 3307, 3308, 3309, 3310, 3318, 3319, 3320, 3321, 3322, 3323, 3324, 3325, 3326, 3327, 3328, 3329, 3330, 3331, 3352, 3353, 3354, 3355, 3356, 3357, 3358, 3359, 3369, 3376, 3377, 3378, 3387, 3388, 3389, 3390, 3391, 3392, 3417, 3418, 3419, 3420, 3425, 3426, 3427, 3451, 3452, 3453, 3454, 3455, 3456, 3457, 3458, 3459, 3460, 3461, 3462, 3507, 3508, 3509, 3510, 3511, 3512, 3524, 3525, 3526, 3527, 3528, 3529, 3530, 3531, 3532, 3533, 3534, 3535]

pass_trim('RSV_F_6APB_A_MODEL_1_famsa3di_aa.fa',
         famsatrim35,
         'RSV_F_6APB_A_MODEL_1_famsa3di_aa_trim35.fa')

foldmasontrim35 = [1330, 1331, 1332, 1333, 1335, 1336, 1338, 1339, 1340, 1341, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1377, 1385, 1386, 1387, 1388, 1389, 1390, 1391, 1392, 1393, 1396, 1397, 1398, 1399, 1400, 1401, 1402, 1403, 1404, 1407, 1408, 1409, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432, 1434, 1435, 1438, 1439, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460, 1461, 1464, 1465, 1466, 1468, 1473, 1474, 1475, 1476, 1477, 1479, 1480, 1481, 1482, 1483, 1486, 1487, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1599, 1600, 1601, 1602, 1603, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1615, 1616, 1617, 1618, 1619, 1620, 1621, 1622, 1623, 1625, 1626, 1635, 1636, 1637, 1638, 1639, 1640, 1641, 1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652, 1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1667, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685, 1686, 1690, 1691, 1692, 1693, 1694, 1696, 1698, 1699, 1700, 1701, 1702, 1704, 1705, 1706, 1707, 1708, 1710, 1715, 1716, 1717, 1718, 1719, 1720, 1721, 1732, 1733, 1734, 1735, 1736, 1737, 1738, 1739, 1740, 1741, 1742, 1743, 1765, 1766, 1767, 1775, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1843, 1844, 1845, 1846, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1864, 1865, 1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1884, 1885, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1906, 1907, 1908, 1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1938, 1942, 1943, 1944, 1945, 1948, 1949, 1950, 1951, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1988, 1989, 1990, 1991, 1992, 1993, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2042, 2044, 2047, 2048, 2049, 2050, 2068, 2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079, 2086, 2087, 2088, 2089, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2103, 2110, 2111, 2113, 2115, 2116, 2117, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2153, 2154, 2155, 2156, 2157, 2158, 2195, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2240, 2241, 2242, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2272, 2273, 2274, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2376, 2377, 2378, 2379, 2380, 2381, 2382, 2383, 2384, 2385, 2386, 2387, 2389, 2390, 2391, 2392, 2393, 2394, 2395, 2404, 2406, 2408, 2409, 2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2421, 2422, 2423, 2424, 2425, 2426, 2427, 2428, 2429, 2430, 2431, 2432, 2433, 2434, 2435, 2438, 2439, 2440, 2441, 2442, 2443, 2444, 2445, 2446, 2447, 2448, 2449, 2450, 2452, 2453, 2456, 2457, 2458, 2459, 2460, 2461, 2462, 2463, 2464, 2465, 2466, 2471, 2472, 2473, 2474, 2475, 2476, 2477, 2478, 2479, 2480, 2481, 2482, 2485, 2487, 2488, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2504, 2505, 2527, 2528, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556, 2557, 2558, 2559, 2560, 2561, 2562, 2569, 2570, 2571, 2572, 2573, 2574, 2575, 2576, 2577, 2578, 2579, 2596, 2597, 2598, 2599, 2600, 2601, 2602, 2603, 2604, 2605, 2606, 2607]

pass_trim('RSV_F_6APB_A_MODEL_1_foldmason_aa.fa',
         foldmasontrim35,
         'RSV_F_6APB_A_MODEL_1_foldmason_aa_trim35.fa')

<br>
<br>

Combine alignments for partitioned phylogenetic inference:

In [7]:
aa_3di_comb('RSV_F_6APB_A_MODEL_1_famsa3di_3di.fa',
            'RSV_F_6APB_A_MODEL_1_famsa3di_aa.fa',
            'RSV_F_6APB_A_MODEL_1_famsa3di_3di_aa.fas')

aa_3di_comb('RSV_F_6APB_A_MODEL_1_foldmason_3di.fa',
            'RSV_F_6APB_A_MODEL_1_foldmason_aa.fa',
            'RSV_F_6APB_A_MODEL_1_foldmason_3di_aa.fas')

In [11]:
aa_3di_comb('RSV_F_6APB_A_MODEL_1_famsa3di_3di_trim35.fas',
            'RSV_F_6APB_A_MODEL_1_famsa3di_aa_trim35.fa',
            'RSV_F_6APB_A_MODEL_1_famsa3di_3di_aa_trim35.fas')

aa_3di_comb('RSV_F_6APB_A_MODEL_1_foldmason_3di_trim35.fas',
            'RSV_F_6APB_A_MODEL_1_foldmason_aa_trim35.fa',
            'RSV_F_6APB_A_MODEL_1_foldmason_3di_aa_trim35.fas')

<br>

Run all trees with `iqtree2` 

`3DI+F+R6` for 3di alignments

`LG+F+R6` for aa alignments

